# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add te import up here.


In [ ]:
import torch
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import time
from torchvision import datasets, transforms, models
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import json

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
import os

# Define data directories
data_dir = 'flowers'
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

print(data_dir)
print(train_dir)
print(valid_dir)
print(test_dir)

In [ ]:
from torchvision import transforms
import os

# Define transforms for the training, validation, and testing sets
train_transform = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the datasets with ImageFolder
# Define data directories
data_dir = 'flowers'
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

print(data_dir)
print(train_dir)
print(valid_dir)
print(test_dir)


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# use the ResNet architecture as recommend above
model = models.resnet50(pretrained=True)
model

In [ ]:

for param in model.parameters():
        param.requires_grad = False
        
classifier = nn.Sequential(
                    nn.Linear(25088, 512),
                    nn.ReLU(),
                    nn.Dropout(p=0.2),
                    nn.Linear(512, 256),
                    nn.ReLU(),
                    nn.Dropout(p=0.2),
                    nn.Linear(256, 102),
                    nn.LogSoftmax(dim = 1)
                )

model.classifier = classifier

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0025)

model.to(device)

In [ ]:
import os
from torchvision import transforms, datasets, models
import torch
import torch.optim as optim
import torch.nn as nn  # Add this line to import the nn module

# TODO: Build and train your network
epochs = 3
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []

# Define transforms for the training, validation, and testing sets
train_transform = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define data directories
data_dir = 'flowers'
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

train_data = datasets.ImageFolder(train_dir, transform=train_transform)
test_data = datasets.ImageFolder(test_dir, transform=test_transform)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transform)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model
model = models.resnet50(pretrained=True)
model.to(device)

# Define the optimizer
optimizer = optim.Adam(model.fc.parameters(), lr=0.0025)

# Define the criterion
criterion = nn.CrossEntropyLoss()  # Add this line to define the criterion

for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            valid_loss = 0
            accuracy = 0
            
            model.eval()
            
            with torch.no_grad():
                for inputs, labels in validloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    valid_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_ps, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            epoch = 0  # Add this line to define the 'epoch' variable
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(valid_loss/len(testloader))
                          
            print(f"Epoch {epoch+1}/{epochs}.. "
                f"Train loss: {running_loss/print_every:.3f}.. "
                f"Test loss: {valid_loss/len(testloader):.3f}.. "
                f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
import time
import torch
# Validate the model on the test set

# Set the model to evaluation mode
model.eval()

# Initialize test accuracy
test_accuracy = 0

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Start validation
start_time = time.time()
print('Validation started.')

# Iterate through the testloader
for images, labels in testloader:
      # Move images and labels to the device
      images, labels = images.to(device), labels.to(device)
      
      # Forward pass
      with torch.no_grad():
            outputs = model(images)
      
      # Calculate accuracy
      _, predicted = torch.max(outputs, 1)
      correct = (predicted == labels).float()
      accuracy = correct.mean()
      test_accuracy += accuracy.item()

# End validation
end_time = time.time()
print('Validation ended.')
validation_time = end_time - start_time
print('Validation time: {:.0f}m {:.0f}s'.format(validation_time // 60, validation_time % 60))

# Calculate and print test accuracy
final_test_accuracy = test_accuracy / len(testloader) * 100
print(f'Test Accuracy: {final_test_accuracy:.2f}%')

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
import os

# Function to save the model checkpoint
def save_model_checkpoint(model, hidden_units, output_units, destination_directory, model_arch, class_to_idx):
    # Create the destination directory if it does not exist
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    # Define model's checkpoint
    model_checkpoint = {
        'model_arch': model_arch,
        'clf_input': 25088,
        'clf_output': output_units,
        'clf_hidden': hidden_units,
        'state_dict': model.state_dict(),
        'model_class_to_index': class_to_idx,
    }

    # Save model in the specified directory
    checkpoint_path = os.path.join(destination_directory, f"{model_arch}_checkpoint.pth")
    torch.save(model_checkpoint, checkpoint_path)
    print(f"{model_arch} checkpoint successfully saved to {checkpoint_path}")

# Define the destination directory

# Define the missing variables
hidden_units = 512
no_output_categories = 10
class_to_idx = {'cat': 0, 'dog': 1}  # Replace with the actual class to index mapping

# Call the save_model_checkpoint function
save_model_checkpoint(model, hidden_units, no_output_categories, destination_directory, 'custom_model', class_to_idx)
destination_directory = 'path/to/destination/directory'

# Call save_model_checkpoint
save_model_checkpoint(model, hidden_units, no_output_categories, destination_directory, 'custom_model', class_to_idx)
no_output_categories = 10  # Replace 10 with the actual number of output categories
save_model_checkpoint(model, hidden_units, no_output_categories, destination_directory, 'custom_model', class_to_idx)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
import torch
import os

# Define the checkpoint file path
checkpoint_path = 'vgg16_bn_checkpoint.pth'

# Function to load checkpoint and rebuild the model
def load_checkpoint(filepath, device):
    # Map the location to GPU if available
    map_location = lambda storage, loc: storage.cuda() if device == "gpu" and torch.cuda.is_available() else 'cpu'

    try:
        # Load the checkpoint
        checkpoint = torch.load(filepath, map_location=map_location)

        # Retrieve information from the checkpoint
        model_arch = checkpoint.get('model_arch')
        clf_input = checkpoint.get('clf_input')
        clf_output = checkpoint.get('clf_output')
        clf_hidden = checkpoint.get('clf_hidden')
        state_dict = checkpoint.get('state_dict')
        class_to_idx = checkpoint.get('model_class_to_index')

        if not all([model_arch, clf_input, clf_output, clf_hidden, state_dict, class_to_idx]):
            raise ValueError("One or more required fields are missing in the checkpoint.")

        return model_arch, clf_input, clf_output, clf_hidden, state_dict, class_to_idx

    except Exception as e:
        print(f"Error loading checkpoint: {e}")
        return None

# Load the checkpoint and rebuild the model
loaded_checkpoint = load_checkpoint(checkpoint_path, 'gpu')

if loaded_checkpoint:
    model_arch, clf_input, clf_output, clf_hidden, state_dict, class_to_idx = loaded_checkpoint

    # Create your model instance with the architecture and other details
    # Example: model = create_custom_model(model_arch, clf_input, clf_output, clf_hidden)

    # Load the model state dict
    model.load_state_dict(state_dict)
    print("Model successfully loaded.")
else:
    print("Failed to load the model.")

# Print the current working directory
print("Current working directory:", os.getcwd())

In [ ]:
checkpoint = torch.load('checkpoint.pth')
state_dict = {k.replace('module.', ''): v for k, v in checkpoint['model_state_dict'].items()}
model.load_state_dict(state_dict)
model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
from PIL import Image
import numpy as np
from torchvision import transforms

# Defines the location of the images to preprocess
practice_img = './flowers/test/19/image_06186.jpg'
# practice_img2 = './flowers/test/70/image_05331.jpg'
# practice_img3 = './flowers/test/100/image_07926.jpg'

# Processes a PIL image for use in a PyTorch model
# Returns results as a numpy array
def process_image(image):
    processed_image = Image.open(image).convert('RGB')  # Open the image
    # Resizes image
    processed_image.thumbnail(size=(256, 256))  # Keeps aspect ratio
    width, height = processed_image.size  # Gets image dimensions

    # Sets new dimensions for center crop
    new_width, new_height = 224, 224
    left = (width - new_width) / 2
    top = (height - new_height) / 2
    right = (width + new_width) / 2
    bottom = (height + new_height) / 2
    processed_image = processed_image.crop((left, top, right, bottom))

    # Converts to tensor and normalizes
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    tensor = transform(processed_image)

    # Convert tensor result to numpy array
    np_processed_image = np.array(tensor)
    return np_processed_image


To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
imshow(process_image('flowers/test/13/image_05745.jpg'))

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
from PIL import Image

def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # Load the image
    image = Image.open(image_path)
    
    # Preprocess the image
    processed_image = process_image(image)
    
    # Convert the processed image to a PyTorch tensor
    tensor = torch.from_numpy(processed_image).unsqueeze(0)
    
    # Move the tensor to the appropriate device (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tensor = tensor.to(device)
    
    # Set the model to evaluation mode
    model.eval()
    
    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass through the model
        output = model(tensor)
        
        # Calculate the probabilities
        probabilities = torch.exp(output)
        
        # Get the top k probabilities and classes
        top_probabilities, top_classes = probabilities.topk(topk, dim=1)
        
        # Convert the tensors to numpy arrays
        top_probabilities = top_probabilities.cpu().numpy()[0]
        top_classes = top_classes.cpu().numpy()[0]
        
        # Convert the class indices to class labels
        class_labels = [model.class_to_idx[str(class_index)] for class_index in top_classes]
        
        return top_probabilities, class_labels

# Define the 'model' variable
model = None

# Example usage
image_path = 'flowers/test/image_04910.jpg'
top_probabilities, class_labels = predict(image_path, model, topk=5)
print(top_probabilities)
print(class_labels)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
from PIL import Image
import matplotlib.pyplot as plt

# Define the image path
image_path = 'path_to_image.jpg' # Replace 'path_to_image.jpg' with the actual path

# Load the image
image = Image.open(image_path)

# Preprocess the image
processed_image = process_image(image)

# Convert the processed image to a PyTorch tensor
tensor = torch.from_numpy(processed_image).unsqueeze(0)

# Move the tensor to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tensor = tensor.to(device)

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation
with torch.no_grad():
    # Forward pass through the model
    output = model(tensor)
    
    # Calculate the probabilities
    probabilities = torch.exp(output)
    
    # Get the top 5 probabilities and classes
    top_probabilities, top_classes = probabilities.topk(5, dim=1)
    
    # Convert the tensors to numpy arrays
    top_probabilities = top_probabilities.cpu().numpy()[0]
    top_classes = top_classes.cpu().numpy()[0]
    
    # Convert the class indices to class labels
    class_labels = [model.class_to_idx[str(class_index)] for class_index in top_classes]
    
    # Get the class names from the class labels
    class_names = [cat_to_name[label] for label in class_labels]
    
    # Display the image
    imshow(processed_image)
    
    # Create a bar graph for the top 5 classes
    plt.figure(figsize=(10, 5))
    plt.barh(class_names, top_probabilities)
    plt.xlabel('Probability')
    plt.ylabel('Class')
    plt.title('Top 5 Classes')
    plt.show()